In [ ]:
!pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install nltk
!pip install gensim

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import gensim
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import torch
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from transformers import BitsAndBytesConfig
import concurrent.futures
from math import log
from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
projects_has_related_projects = pd.read_csv('./projects_has_releted_projects.csv')

In [ ]:
related_projects = np.concatenate([pd.unique(projects_has_related_projects['project']), pd.unique(projects_has_related_projects['subject'])])

In [ ]:
def get_projects_data(project_id):
    sparql_query = """
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX arkc: <https://xxx.xxx.ie/Ontologies/ARKCube#>
      PREFIX arkp: <https://xxx.xxx.ie/Ontologies/ARKPlatform#>
      SELECT DISTINCT
        ?field
        ?stage
        ?textualDescription
        ?concept
        WHERE {
        <%(subject)s> a arkc:Project .
        BIND(<%(subject)s> AS ?project)
        ?project arkc:name ?projectName .
        {
        # ?pa does not have text or concept
        ?project arkc:hasProjectAnalysis ?range .
        ?range a ?field .
        ?range arkc:describesStage ?stage .
        BIND(?range AS ?subject)
        ?subject ?predicate ?object .
        OPTIONAL {
            ?range arkc:textualDescription ?textualDescription .
        }

        OPTIONAL {
        ?range arkc:hasRelatedConcept ?concept .
        }
        } UNION
        {
        ?project arkc:hasProjectAnalysis ?pa .
        ?pa arkc:describesStage ?stage .
        ?pa (arkc:hasPotentialLoss | arkc:hasProjectedGain | arkc:hasProjectedReliabilityOfGain | arkc:hasReliabilityOfGain | arkc:hasGainAchieved) ?range .
        ?range a ?field .


        BIND(?range AS ?subject)


        ?subject ?predicate ?object .
        OPTIONAL {
        ?range arkc:textualDescription ?textualDescription .
        }

        OPTIONAL {
        ?range arkc:hasRelatedConcept ?concept .
        }
        } UNION
        {
        ?project arkc:hasProjectAnalysis ?pa .
        ?pa arkc:describesStage ?stage .
        ?pa (arkc:hasOutcome | arkc:hasMechanism | arkc:hasContext) ?range .
        ?range a ?field .


        BIND(?range AS ?subject)


        ?subject ?predicate ?object .
        OPTIONAL {
        ?range arkc:textualDescription ?textualDescription .
        }

        OPTIONAL {
        ?range arkc:hasRelatedConcept ?concept .
        }
        } UNION
        {
        ?project arkc:hasQuestionnaire/^arkc:isQuestionOf ?question .
        ?question arkc:describesStage ?stage .
        ?question arkc:hasAspect ?aspect .
        ?question arkc:hasDimension ?dimension .
        ?question arkc:hasAnswer ?range .
        ?range a ?field .
        BIND(?range AS ?subject)
        ?subject ?predicate ?object .
        ?range arkc:textualDescription ?textualDescription .


        OPTIONAL {
        ?range arkc:hasRelatedConcept ?concept .
        }


        } UNION
        {
        ?project arkc:hasQuestionnaire ?questionnaire .
        ?questionnaire arkc:hasQuestion ?question .
        ?question arkc:describesStage ?stage .
        ?question arkc:hasAspect ?aspect .
        ?question arkc:hasDimension ?dimension .
        ?question arkc:hasAnswer ?range .
        ?range a ?field .


        BIND(?range AS ?subject)


        ?subject ?predicate ?object .
        ?range arkc:textualDescription ?textualDescription .


        OPTIONAL {
        ?range arkc:hasRelatedConcept ?concept .
        }


        }
        }
  """

    sparql = SPARQLWrapper("https://stagingark.xxx.ie/fuseki/ark_data/query")
    sparql.setQuery(sparql_query % {'subject': project_id})
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    df = pd.DataFrame(results["results"]["bindings"])
    df.dropna(axis=0,how='any',inplace=True)

    if 'field' in df.columns:
        df['field'] = df['field'].apply(lambda x: x['value'])
    if 'stage' in df.columns:
        df['stage'] = df['stage'].apply(lambda x: x['value'])
    if 'textualDescription' in df.columns:
        df['textualDescription'] = df['textualDescription'].apply(lambda x: x['value'])
    if 'concept' in df.columns:
        df['concept'] = df['concept'].apply(lambda x: x['value'])
    return df

In [ ]:
project_dfs = {}
for item in np.unique(related_projects):
    project_dfs[item] = get_projects_data(item)

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
def text_clear(text):
  text= text.lower()
  line = re.sub(r"[^A-Za-z0-9()<>:;,.!?\'\"]"," ",text)
  line = re.sub(r","," ",text)
  line = re.sub(r";"," ",line)
  line = re.sub(r"\'"," ",line)
  line = re.sub(r":"," ",line)
  line = re.sub(r"!"," ",line)
  line = re.sub(r"\."," ",line)
  line = re.sub(r"\/"," ",line)
  line = re.sub(r"\?"," ", line)
  #line = re.sub(r"[\\d]"," ",line)
  line = re.sub(r"\'\'"," ",line)
  line = re.sub(r"\("," ",line)
  line = re.sub(r"\)"," ",line)
  line = re.sub(r"\<"," ",line)
  line = re.sub(r"\>"," ",line)
  line = re.sub(r"  "," ",line)
  line = re.sub(r"\n","",line)
  line = re.sub(r"\t"," ",line)
  text = line.strip()
  punctuationfree="".join([i for i in text if i not in string.punctuation])
  tokens = re.split('W+',punctuationfree)
  output= [wordnet_lemmatizer.lemmatize(porter_stemmer.stem(i)) for i in tokens if i not in stop_words]
  output = "".join([i for i in output])
  return output

In [ ]:
def embeddings_similarity_score(embeddings_A, embeddings_B):
  embeddings_A = embeddings_A.flatten().reshape(1,-1)
  embeddings_B = embeddings_B.flatten().reshape(1,-1)
  if embeddings_A.shape[1] > embeddings_B.shape[1]:
    num_zero_columns = embeddings_A.shape[1] - embeddings_B.shape[1]
    embeddings_B = np.pad(embeddings_B, ((0, 0), (0, num_zero_columns)), mode='constant')
  else:
    num_zero_columns = embeddings_B.shape[1] - embeddings_A.shape[1]
    embeddings_A = np.pad(embeddings_A, ((0, 0), (0, num_zero_columns)), mode='constant')

  cosine_similarity_score = cosine_similarity(embeddings_A,embeddings_B)
  return cosine_similarity_score[0]

In [ ]:
def similarity_score_in_paris(embeddings_A, embeddings_B):
    similarity_score_list = []
    if isinstance(embeddings_A, dict) and isinstance(embeddings_B, dict):
    #if type(embeddings_A) == dict:
        for key_1, value_1 in embeddings_A.items():
            for key_2, value_2 in embeddings_B.items():
                similarity_score_list.append(embeddings_similarity_score(value_1,value_2))
    elif isinstance(embeddings_A, torch.Tensor) and isinstance(embeddings_B, torch.Tensor):
    #if type(embeddings_A) == torch.Tensor:
        embeddings_A = embeddings_A.cpu().detach().numpy()
        embeddings_B = embeddings_B.cpu().detach().numpy()
        for idx_1, value_1 in enumerate(embeddings_A):
            for idx_2, value_2 in enumerate(embeddings_B):
                similarity_score_list.append(embeddings_similarity_score(value_1,value_2))
    elif isinstance(embeddings_A, np.ndarray) and isinstance(embeddings_B, np.ndarray):
        for idx_1, value_1 in enumerate(embeddings_A):
            for idx_2, value_2 in enumerate(embeddings_B):
                similarity_score_list.append(embeddings_similarity_score(value_1,value_2))

    return np.mean(similarity_score_list)

word2vec

In [ ]:
def concept_word2vec_embeddings(concept_list):
  concepts, concept_token = [], []
  concept_embeddings_d, token_d = {}, {}
  for c in concept_list:
    tmp = c.split("#")[-1]
    tmp = re.findall('[A-Z][^A-Z]*',tmp)
    tmp = " ".join(tmp)
    concepts.append(tmp)
    t = nltk.word_tokenize(tmp.lower())
    token_d[tmp]=t
    concept_token.append(t)
  model = gensim.models.Word2Vec(concept_token, min_count=1,vector_size=100, window=5)

  for concept in concepts:
    #token = nltk.word_tokenize(concept.lower())
    embeddings = [model.wv[w_t] for w_t in token_d[concept]]
    concept_embeddings = np.concatenate(embeddings, axis=0)
    #concept_embeddings = np.mean(embeddings, axis=0)
    concept_embeddings_d[concept] = concept_embeddings

  return concept_embeddings_d

In [ ]:
def words_embeddings(text_list):
    concept_cube_text, concept_cube_text_token = [], []
    concept_cube_text_embeddings_d = {}
    for t in text_list:
        concept_cube_text.append(t)
        concept_cube_text_token.append(nltk.word_tokenize(t.lower()))
    model = gensim.models.Word2Vec(concept_cube_text_token, min_count=1,vector_size=100, window=5)
    for c_t in concept_cube_text:
        c_t_token = nltk.word_tokenize(c_t.lower())
        embeddings = [model.wv[w_t] for w_t in c_t_token]
        concept_cube_text_embeddings = np.mean(embeddings, axis=0)
        #concept_cube_text_embeddings = np.concatenate(embeddings, axis=0)
        concept_cube_text_embeddings_d[c_t] = concept_cube_text_embeddings
    return concept_cube_text_embeddings_d

concept

In [ ]:
projects_concepts_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        concepts = value[value['concept'].notnull()]['concept'].unique()
        projects_concepts_embeddings[key] = concept_word2vec_embeddings(concepts)
    else:
      print(key)

In [ ]:
elements = projects_concepts_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
results_concept= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-43-c78c2f186e3b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_concept= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
results_concept.to_csv("./results/results_concept_only_word2vec_mean.csv", index=False)

In [ ]:
results_concept.to_csv("./results/results_concept_only_word2vec_concatenate.csv", index=False)

cube text

In [ ]:
cube_text_embeddings = {}
for key, value in project_dfs.items():
    if 'textualDescription' in value.columns:
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        words = value[value['clearedTextualDescription'].notnull()]['clearedTextualDescription'].unique()
        cube_text_embeddings[key] = words_embeddings(words)
    else:
        print(key)

In [ ]:
elements = cube_text_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_projects = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
result_words= df_projects.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-59-776d17cf0708>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_words= df_projects.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
result_words.to_csv("./results/results_cube_text_only_word2vec_mean.csv", index=False)

In [ ]:
result_words.to_csv("./results/results_cube_text_only_word2vec_concatenate.csv", index=False)

concepts and cube text

In [ ]:
projects_words_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns and 'textualDescription' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        value['cube_text'] = value['concept_terms']+' '+value['clearedTextualDescription']
        projects_words_embeddings[key] = words_embeddings(value['cube_text'])

In [ ]:
elements = projects_words_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_projects_words = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
result_words= df_projects_words.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-78-72e0c6853dd2>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_words= df_projects_words.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
result_words.to_csv("./results/results_cube_text_and_concept_word2vec_mean.csv", index=False)

In [ ]:
result_words.to_csv("./results/results_cube_text_and_concept_word2vec_concatenate.csv", index=False)

LLAMa

In [ ]:
access_token = "xxxxxx"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

t = LlamaTokenizer.from_pretrained(model_id, token = access_token)
t.pad_token = t.eos_token
m = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", token = access_token, quantization_config=bnb_config)
m.eval()

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=

In [ ]:
def embedding_generation_with_llama(text):
    t_input = t(text, padding=True, return_tensors="pt").to(device)


    with torch.no_grad():
        last_hidden_state = m(**t_input, output_hidden_states=True).hidden_states[-1]


    weights_for_non_padding = t_input.attention_mask * torch.arange(start=1, end=last_hidden_state.shape[1] + 1).unsqueeze(0).to(device)


    sum_embeddings = torch.sum(last_hidden_state * weights_for_non_padding.unsqueeze(-1), dim=1)
    num_of_none_padding_tokens = torch.sum(weights_for_non_padding, dim=-1).unsqueeze(-1)
    sentence_embeddings = sum_embeddings / num_of_none_padding_tokens

    if torch.isinf(sentence_embeddings).any() or not torch.isfinite(sentence_embeddings).all():
        sentence_embeddings = sentence_embeddings.float()
        finite_value = torch.tensor(1e10, dtype=torch.float64)
        cleaned_embeddings = torch.where(torch.isfinite(sentence_embeddings), sentence_embeddings, finite_value)
        sentence_embeddings = cleaned_embeddings

    return sentence_embeddings

concept

In [ ]:
projects_concepts_llama_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        concepts = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        concepts = concepts.unique()
        concepts = pd.Series(concepts).astype(str).tolist()
        projects_concepts_llama_embeddings[key] = embedding_generation_with_llama(concepts)

In [ ]:
elements = projects_concepts_llama_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_concepts = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
result_concepts= df_results_concepts.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

In [ ]:
result_concepts.to_csv("./results_8_projects/results_concepts_llama.csv", index=False)

cube_text

In [ ]:
projects_cube_text_embeddings = {}
for key, value in project_dfs.items():
    if 'textualDescription' in value.columns:
        cube_text = value[(value['textualDescription'].notnull())]['textualDescription'].unique()
        cube_text = pd.Series(cube_text).astype(str).apply(lambda x:text_clear(x)).tolist()
        projects_cube_text_embeddings[key] = embedding_generation_with_llama(cube_text)
        torch.cuda.empty_cache()

In [ ]:
elements = projects_cube_text_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_cube_text = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
result_words = df_results_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

In [ ]:
result_words.to_csv("./results_8_projects/results_cube_text_llama.csv", index=False)

concepts and cube text

In [ ]:
projects_cube_text_concept_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        value['cube_text'] = value['concept_terms']+' '+value['clearedTextualDescription']
        words = value[value['cube_text'].notnull()]['cube_text'].unique().tolist()
        projects_cube_text_concept_embeddings[key] = embedding_generation_with_llama(words)

In [ ]:
elements = projects_cube_text_concept_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_concept_cube_text = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_cube_text_results= df_results_concept_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

In [ ]:
concept_cube_text_results.to_csv("./results_cube_text_and_concept_llama.csv", index=False)

MiniLM

In [ ]:
def miniLM_embeddings(text):

  model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  embeddings = model.encode(text)

  return embeddings

concept

In [ ]:
projects_concepts_miniLM_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        concepts = value[value['concept'].notnull()]['concept']
        concepts = pd.Series(concepts).astype(str).tolist()
        projects_concepts_miniLM_embeddings[key] = miniLM_embeddings(concepts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in tra

In [ ]:
elements = projects_concepts_miniLM_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_concept = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_results= df_results_concept.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-84-2707c547a14b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_results= df_results_concept.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_results.to_csv("./results/results_concept_miniLM.csv", index=False)

cube_text

In [ ]:
cube_text_embeddings = {}
for key, value in project_dfs.items():
    if 'textualDescription' in value.columns:
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        text = value[value['clearedTextualDescription'].notnull()]['clearedTextualDescription'].to_list()
        cube_text_embeddings[key] = miniLM_embeddings(text)
    else:
        print(key)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
elements = cube_text_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_cube_text = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
cube_text_results= df_results_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-96-bd4d3ab4fd56>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cube_text_results= df_results_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
cube_text_results.to_csv("./results/results_cube_text_miniLM.csv",index=False)

concept and cube text

In [ ]:
projects_cube_text_concept_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        value['cube_text'] = value['concept_terms']+' '+value['clearedTextualDescription']
        words = value[value['cube_text'].notnull()]['cube_text'].to_list()
        projects_cube_text_concept_embeddings[key] = miniLM_embeddings(words)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
elements = projects_cube_text_concept_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_concept_cube_text = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_cube_text_results= df_results_concept_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-100-d06fff7f8903>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_cube_text_results= df_results_concept_cube_text.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_cube_text_results.to_csv("./results/results_cube_text_concept_miniLM.csv", index=False)

Bert

In [ ]:
def Bert_embeddings(text):

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  model = BertModel.from_pretrained('bert-base-uncased')

  encoding = tokenizer.batch_encode_plus(
      text,
      padding = True,
      truncation = True,
      return_tensors = 'pt',
      add_special_tokens=True

  )
  input_ids = encoding['input_ids']
  attention_mask = encoding['attention_mask']

  with torch.no_grad():
    outputs = model(input_ids,attention_mask=attention_mask)
    words_embeddings = outputs.last_hidden_state

  return words_embeddings

concept

In [ ]:
projects_concepts_Bert_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        concepts = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        concepts = pd.Series(concepts).astype(str).tolist()
        projects_concepts_Bert_embeddings[key] = Bert_embeddings(concepts)

In [ ]:
elements = projects_concepts_Bert_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results_concept = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_results= df_results_concept.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-117-2707c547a14b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_results= df_results_concept.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_results.to_csv("./results/results_concept_Bert.csv", index=False)

cube text

In [ ]:
cube_text_embeddings = {}
for key, value in project_dfs.items():
    if 'textualDescription' in value.columns:
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        text = value[value['clearedTextualDescription'].notnull()]['clearedTextualDescription'].unique()
        cube_text_embeddings[key] = Bert_embeddings(text.tolist())
    else:
        print(key)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
elements = cube_text_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-121-bae4875fe4d8>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
cube_text_results.to_csv("./results/results_cube_text_Bert.csv", index=False)

concept and cube text

In [ ]:
projects_cube_text_concept_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        value['cube_text'] = value['concept_terms']+' '+value['clearedTextualDescription']
        words = value[value['cube_text'].notnull()]['cube_text'].unique()
        projects_cube_text_concept_embeddings[key] = Bert_embeddings(words)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
elements = projects_cube_text_concept_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-125-bc47e32c871e>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_Bert.csv", index=False)

Combine Bert generated embeddings and miniLM generated embeddings

In [ ]:
def MultiEmb(terms, sentences, alpha, n_components):

  #torch.manual_seed(42)

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  model = BertModel.from_pretrained('bert-base-uncased')

  encoding = tokenizer.batch_encode_plus(
      terms,
      padding = True,
      truncation = True,
      return_tensors = 'pt',
      add_special_tokens=True

  )
  input_ids = encoding['input_ids']
  attention_mask = encoding['attention_mask']

  #[batch_size, seq_len, 768]
  with torch.no_grad():
    outputs = model(input_ids,attention_mask=attention_mask)
    words_embeddings = outputs.last_hidden_state

  sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  #sentence_model.eval()
  #[batch_size, embedding_dim]
  sentence_embeddings = sentence_model.encode(sentences)

  sentence_embeddings = torch.tensor(sentence_embeddings).unsqueeze(1)

  torch.manual_seed(42)
  linear_layer = torch.nn.Linear(768, 384)

  words_embeddings_reduced_dim = linear_layer(words_embeddings)

  repeated_sentence_embedding = sentence_embeddings.repeat(1, words_embeddings_reduced_dim.size(1), 1)

  combined_embeddings = torch.cat((words_embeddings_reduced_dim, repeated_sentence_embedding), dim=-1)


  weighted_sum_embeddings = (alpha * words_embeddings_reduced_dim) + ((1 - alpha) * repeated_sentence_embedding)  # [batch_size, seq_len, 384]

  batch_size, seq_len, _ = combined_embeddings.size()
  combined_embeddings_reshaped = combined_embeddings.view(-1, combined_embeddings.size(-1))  # Shape: [batch_size*seq_len, 768]
  pca = PCA(n_components=n_components)
  combined_embeddings_pca = pca.fit_transform(combined_embeddings_reshaped.detach().numpy())  # Shape: [batch_size*seq_len, n_components]
  # Reshape back to original batch and seq_len
  combined_embeddings_pca = torch.tensor(combined_embeddings_pca).view(batch_size, seq_len, n_components)  # Shape: [batch_size, seq_len, n_components]

  return  combined_embeddings, weighted_sum_embeddings, combined_embeddings_pca

In [ ]:
projects_cube_text_concept_Multi_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns and 'textualDescription' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        text = value[value['clearedTextualDescription'].notnull()]['clearedTextualDescription'].to_list()
        projects_cube_text_concept_Multi_embeddings[key] = MultiEmb(value['concept_terms'], text, 0.1, 16)[1]

In [ ]:
elements = projects_cube_text_concept_Multi_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-173-bc47e32c871e>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_multiEmb.csv", index=False)

In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_multiEmb_weighted_sum.csv", index=False)

In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_multiEmb_pca.csv", index=False)

Combine Word2Vec generated embeddings and miniLM generated embeddings

In [ ]:
def MegWord2MiniEmb(terms, sentences, alpha, n_components):

  concepts, concept_token, words_embeddings = [], [], []
  token_d = {}
  for tmp in terms:
    t = nltk.word_tokenize(tmp.lower())
    token_d[tmp]=t
    concept_token.append(t)
  model = gensim.models.Word2Vec(concept_token, min_count=1,vector_size=100, window=5)

  for term in terms:
    embeddings = [model.wv[w_t] for w_t in token_d[term]]
    #concept_embeddings = np.concatenate(embeddings, axis=0)
    concept_embeddings = np.mean(embeddings, axis=0)
    words_embeddings.append(concept_embeddings)

  words_embeddings = torch.tensor(words_embeddings).unsqueeze(1)
  sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

  #[batch_size, embedding_dim]
  sentence_embeddings = sentence_model.encode(sentences)

  sentence_embeddings = torch.tensor(sentence_embeddings).unsqueeze(1)
  torch.manual_seed(42)
  linear_layer = torch.nn.Linear(100, 384)

  words_embeddings_reduced_dim = linear_layer(words_embeddings)

  #linear_layer_sentence = torch.nn.Linear(768, 384)
  #sentence_embeddings_reduced_dim = linear_layer_sentence(sentence_embeddings)

  repeated_sentence_embedding = sentence_embeddings.repeat(1, words_embeddings_reduced_dim.size(1), 1)

  combined_embeddings = torch.cat((words_embeddings_reduced_dim, repeated_sentence_embedding), dim=-1)

  weighted_sum_embeddings = (alpha * words_embeddings_reduced_dim) + ((1 - alpha) * repeated_sentence_embedding)

  batch_size, seq_len, _ = combined_embeddings.size()
  combined_embeddings_reshaped = combined_embeddings.view(-1, combined_embeddings.size(-1))
  pca = PCA(n_components=n_components)
  combined_embeddings_pca = pca.fit_transform(combined_embeddings_reshaped.detach().numpy())

  combined_embeddings_pca = torch.tensor(combined_embeddings_pca).view(batch_size, seq_len, n_components)
  print(combined_embeddings_pca.shape)
  return  combined_embeddings, weighted_sum_embeddings, combined_embeddings_pca

In [ ]:
projects_cube_text_concept_Multi_embeddings = {}
for key, value in project_dfs.items():
    if 'concept' in value.columns and 'textualDescription' in value.columns:
        value['concept_terms'] = value['concept'].map(lambda x: " ".join(re.findall('[A-Z][^A-Z]*',x.split("#")[-1])))
        value['clearedTextualDescription'] = value['textualDescription'].map(lambda x:text_clear(x))
        text = value[value['clearedTextualDescription'].notnull()]['clearedTextualDescription'].to_list()
        projects_cube_text_concept_Multi_embeddings[key] = MegWord2MiniEmb(value['concept_terms'], text, 0.1, 6)[2]

In [ ]:
elements = projects_cube_text_concept_Multi_embeddings

def similarity_score_caculate_pairs(pair):
    key1, key2 = pair
    return (key1, key2, similarity_score_in_paris(elements[key1], elements[key2]))


pairs = [(key1, key2) for key1 in elements.keys() for key2 in elements.keys() if key1 != key2]


with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(similarity_score_caculate_pairs, pairs))

df_results = pd.DataFrame(results, columns=['project1', 'project2', 'Similarity_Score'])

In [ ]:
concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])

<ipython-input-304-bc47e32c871e>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concept_cube_text_results= df_results.sort_values(by="Similarity_Score",ascending=False).groupby("project1", group_keys=True).apply(lambda a: a[:])


In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_megWord2MiniEmb.csv", index=False)

In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_megWord2MiniEmb_weighted_sum.csv", index=False)

In [ ]:
concept_cube_text_results.to_csv("./results/results_concept_cube_text_megWord2MiniEmb_pca.csv", index=False)

Evluation

In [ ]:
def evluation(k, y_true_dict, similar_projects_dict):
  invalid_projects = []
  # Compute metrics
  precisions, recalls, ndcgs, hits, map_scores = [], [], [], [], []
  project_idxs = list(y_true_dict.keys())

  for pid in project_idxs:
      if pid not in similar_projects_dict or len(similar_projects_dict[pid]) < k:
          invalid_projects.append(pid)
          continue
      pred_list, rel_set = similar_projects_dict[pid][:k], y_true_dict[pid]

      pred_real = "pid:"+str(pid)+' '+"pred_list:"+str(pred_list)+' '+"rel_set:"+str(rel_set)

      if len(pred_list) == 0:
          continue


      dcg = 0.0
      hit_num = 0.0
      for i in range(len(pred_list)):
          if pred_list[i] in rel_set:
              dcg += 1. / (log(i + 2) / log(2))
              hit_num += 1
      # idcg
      idcg = 0.0
      for i in range(min(len(rel_set), len(pred_list))):
          idcg += 1. / (log(i + 2) / log(2))
      ndcg = dcg / idcg
      recall = hit_num / len(rel_set)
      precision = hit_num / len(pred_list)
      hit = 1.0 if hit_num > 0.0 else 0.0

      #map
      map_score = 0.0
      num_hits = 0.0
      score = 0.0
      for i,p in enumerate(pred_list):
          if p in rel_set and p not in pred_list[:i]:
              num_hits+=1.0
              score+=num_hits/(i+1.0)
      map_score = score/min(len(rel_set),k)
      #map_score = score / min(len(rel_set), len(pred_list)) if len(rel_set) > 0 else 0.0


      ndcgs.append(ndcg)
      recalls.append(recall)
      precisions.append(precision)
      hits.append(hit)
      map_scores.append(map_score)

  avg_precision = np.mean(precisions) * 100
  avg_recall = np.mean(recalls) * 100
  avg_ndcg = np.mean(ndcgs) * 100
  avg_hit = np.mean(hits) * 100
  avg_map = np.mean(map_scores) * 100

  #tmp = 'map: '+str(avg_map)+' '+'ndcg: '+str(avg_ndcg)+ ' '+'recall: '+str(avg_recall)+' '+'precision: '+str(avg_precision)+' '+str(len(invalid_projects))

  print("invalid projects:", str(len(invalid_projects)))
  print('MAP={:.3f} | NDCG={:.3f} |  Recall={:.3f} | Precision={:.3f} | Hits={:.3f}'.format(
          avg_map, avg_ndcg, avg_recall, avg_precision, avg_hit))

In [ ]:
y_true_dict = projects_has_related_projects.groupby('project')['subject'].apply(list).to_dict()

In [ ]:
threshold = 0

In [ ]:
results = pd.read_csv("./results/results_concept_only_word2vec_mean.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=0.000 | NDCG=0.000 |  Recall=0.000 | Precision=0.000 | Hits=0.000


In [ ]:
results = pd.read_csv("./results/results_concept_only_word2vec_concatenate.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=25.000 | NDCG=25.000 |  Recall=25.000 | Precision=25.000 | Hits=25.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_only_word2vec_mean.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=0.000 | NDCG=0.000 |  Recall=0.000 | Precision=0.000 | Hits=0.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_only_word2vec_concatenate.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=25.000 | NDCG=25.000 |  Recall=25.000 | Precision=25.000 | Hits=25.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_and_concept_word2vec_mean.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=0.000 | NDCG=0.000 |  Recall=0.000 | Precision=0.000 | Hits=0.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_and_concept_word2vec_concatenate.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=25.000 | NDCG=25.000 |  Recall=25.000 | Precision=25.000 | Hits=25.000


In [ ]:
results = pd.read_csv("./results/results_concepts_llama.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=50.000 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_llama.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=37.500 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_and_concept_llama.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=37.500 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_concept_miniLM.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=0.000 | NDCG=0.000 |  Recall=0.000 | Precision=0.000 | Hits=0.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_miniLM.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=100.000 | NDCG=100.000 |  Recall=87.500 | Precision=100.000 | Hits=100.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_concept_miniLM.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=100.000 | NDCG=100.000 |  Recall=87.500 | Precision=100.000 | Hits=100.000


In [ ]:
results = pd.read_csv("./results/results_concept_Bert.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=75.000 | NDCG=75.000 |  Recall=75.000 | Precision=75.000 | Hits=75.000


In [ ]:
results = pd.read_csv("./results/results_cube_text_Bert.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=50.000 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_Bert.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=50.000 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_multiEmb.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=75.000 | NDCG=75.000 |  Recall=75.000 | Precision=75.000 | Hits=75.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_multiEmb_weighted_sum.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=75.000 | NDCG=75.000 |  Recall=75.000 | Precision=75.000 | Hits=75.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_multiEmb_pca.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=50.000 | Precision=50.000 | Hits=50.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_megWord2MiniEmb.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=100.000 | NDCG=100.000 |  Recall=87.500 | Precision=100.000 | Hits=100.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_megWord2MiniEmb_weighted_sum.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=100.000 | NDCG=100.000 |  Recall=87.500 | Precision=100.000 | Hits=100.000


In [ ]:
results = pd.read_csv("./results/results_concept_cube_text_megWord2MiniEmb_pca.csv")
results_dict = results[results['Similarity_Score'] > threshold].groupby('project1')['project2'].apply(list).to_dict()
evluation(1,y_true_dict,results_dict)

invalid projects: 0
MAP=50.000 | NDCG=50.000 |  Recall=50.000 | Precision=50.000 | Hits=50.000
